In [ ]:
!pip install pytorch-lightning
!pip install torch-fidelity

In [4]:
import os
import csv
import torch
import torchvision
from torchvision import transforms
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pytorch_lightning as pl


#Make your network
here we consider a FFNN with two hidden layers, one input and one output
the activation func is a sigmoid function.
We consider that we want to create a linear regression. So, we have one input as x, and the output as y values

In [5]:
class Net(nn.Module):
  def __init__(self,Ni=1, Nh1=128, Nh2=256, No=1):
    super().__init__()

    print("network initializaiton")

    self.fc1 = nn.Linear(Ni,Nh1)
    self.fc2 = nn.Linear(Nh1,Nh2)
    self.out = nn.Linear(Nh2,No)
    self.act = nn.Sigmoid()

  def forward(self,x,additional_out = False):
    x = self.act(self.fc1(x))
    x = self.act(self.fc2(x))
    x = self.out(x)

    return x

#Import and process the data
Consider we have an excel data with the x column as input and the y column as the output or the labels.
We have to do three things:
1. reading teh data
2. convert the data to the tensors
3. load the in the batches using the dataloader. The suggested value for the batch number is a number like: $ 2^n$
for example the numbers like 2, 4, 8, 16, 32, 64, 128, ... are good numbers for the batch size.




In [6]:

class CsvDataset(Dataset):
  def __init__(self,csv_file,transform = None):
    self.transform = transform
    self.data = []
    with open(csv_file, newline='') as csvfile:
      lines = csvfile.read().split()

    for line in lines:
      sample = line.split(',')
      if (sample[0]) != 'x'and  len(sample) >= 2    :
        self.data.append((float(sample[0]),float(sample[1])))

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    sample = self.data[idx]
    if self.transform:
      sample = self.transform(sample)
    return sample

In [7]:
class ToTensor(object):
  def __call__(self,sample):
    x,y = sample
    return(torch.tensor(x).float(),
           torch.tensor(y).float())

In [8]:
composed_transform = transforms.Compose([ToTensor()])

train_dataset = CsvDataset('train.csv',transform=composed_transform)
val_dataset = CsvDataset('validation.csv',transform=composed_transform)

#Dataloader
for the dataloader:

enable shuffling (needed only fro train data_
try different values for batch size

In [66]:
train_dataloader = DataLoader(train_dataset,batch_size = 1024,shuffle=True,num_workers=0)
val_dataloader = DataLoader(val_dataset,batch_size = len(val_dataset),shuffle=False,num_workers=0)

#Choose the GPU device is available
 Check if the GPU is available


In [21]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

Training device: cuda


# Pytorch lightening

In [63]:
class LitFNN(pl.LightningModule):
  def __init__(
        self,
        lr: float = 1e-3,
        b1: float = 0.5,
        b2: float = 0.999

    ):
    super().__init__()
    self.save_hyperparameters()
    #network
    self.net = Net()

    #define loss
    self.loss_fn = nn.MSELoss()


    #forward path
  def forward(self, x):
      return self.net(x)

  #config the optimizer
  def configure_optimizers(self):
      lr = self.hparams.lr
      b1 = self.hparams.b1
      b2 = self.hparams.b2

      optimizer = torch.optim.Adam(self.net.parameters(), lr=lr, betas=(b1, b2))

      return optimizer

  #training step
  def training_step(self, train_batch, batch_idx):
      x_batch = train_batch[0].unsqueeze(0)
      x_batch = torch.transpose(x_batch,0,1)
      label_batch = train_batch[1].unsqueeze(0)
      label_batch = torch.transpose(label_batch,0,1)
      out = self.net(x_batch)
      loss = self.loss_fn(out,label_batch)
      self.log("train_loss",loss, prog_bar=True)
      return {'loss': loss}
  #validation step
  def validation_step(self,valid_batch,batch_idx):
      x_batch = valid_batch[0].unsqueeze(0)
      x_batch = torch.transpose(x_batch,0,1)
      label_batch = valid_batch[1].unsqueeze(0)
      label_batch = torch.transpose(label_batch,0,1)
      out = self.net(x_batch)
      loss = self.loss_fn(out,label_batch)
      self.log("val_loss",loss, prog_bar=True)
      return {'validation_loss': loss}

In [92]:
import json
from pytorch_lightning.callbacks import Callback


class History(Callback):

    def __init__(
        self,

        dirpath:str ='/content/',
        filename:str = 'history.json',

    ):
        super().__init__()
        self.t_loss = []
        self.v_loss = []
        self.dirpath = dirpath
        self.filename = filename
        self.path = os.path.join(dirpath,filename)
        self._init_dict()
        self._init_lists()

    def _init_dict(self):
        self.dict = {'train_loss':[],
                     'val_loss':[]}
        os.makedirs(self.dirpath, exist_ok=True)
        with open(self.path,'w') as f:
            json.dump(self.dict,f)

    def _init_lists(self):
        self.t_loss = []
        self.v_loss = []


    def on_train_batch_end(self,trainer, pl_module,outputs,batch,batch_idx,unused=0):
        self.t_loss.append(outputs['loss'].item())


    def on_validation_batch_end(self,trainer, pl_module,outputs,batch,batch_idx):
        self.v_loss.append(outputs['validation_loss'].item())


    def on_train_epoch_end(self,trainer,pl_module):
        self.dict['train_loss'].append(float(torch.mean(torch.tensor(self.t_loss))))
        with open(self.path,'w') as f:
            json.dump(self.dict,f)
        self.t_loss = []


    def on_validation_epoch_end(self, trainer, pl_module):
        self.dict['val_loss'].append(float(np.mean(self.v_loss)))
        with open(self.path,'w') as f:
            json.dump(self.dict,f)
        self.v_loss = []


In [ ]:
experiment_dir = "/content"
call_back = History(dirpath ="/content")

trainer = pl.Trainer(default_root_dir=experiment_dir,
                     #gpus=1,
                     precision=16,
                     max_epochs=100,
                     val_check_interval=1, #check_validation_accuracy
                     callbacks=[call_back],
                     num_sanity_val_steps=0,
                     #logger=logger,
                     )
net = LitFNN()
trainer.fit(net, train_dataloader,val_dataloader)